<a href="https://colab.research.google.com/github/kahhotan/Coding-Projects/blob/main/pyspark_guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyspark basic operations

In [1]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 51.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=66f151e51a8760c2366e9a2d73485d970a79f829ae721612daaf0473a0bc6f28
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
#create session
spark=SparkSession.builder.appName('Practice').getOrCreate()

In [4]:
spark

In [5]:
#load dataset
df_pyspark=spark.read.csv('/content/sample_data/test1.csv')

In [6]:
type(df_pyspark)
#pyspark is sql dataframe

pyspark.sql.dataframe.DataFrame

In [7]:
df_pyspark.show()
#issue: Name, age, etc is supposed to be main header

+---------+---+----------+------+
|      _c0|_c1|       _c2|   _c3|
+---------+---+----------+------+
|     Name|age|Experience|Salary|
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [8]:
#reload dataset with .option to make first row the header
df_pyspark=spark.read.option('header','true').csv('/content/sample_data/test1.csv')

In [9]:
df_pyspark.show()
#now we have fixed the headers

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [10]:
df_pyspark.printSchema()
#issue: numerics are read as str instead of int. by default, pyspark will read all features as str unless specified

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [11]:
#reload dataset with .option and inferSchema = 'True'
df_pyspark=spark.read.option('header','true').csv('/content/sample_data/test1.csv',inferSchema='True')

In [12]:
#checking schema again
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [13]:
#alternative method of loading dataset
df_pyspark=spark.read.csv('/content/sample_data/test1.csv',header='True',inferSchema='True')

In [14]:
#exploring feature header
df_pyspark.columns

['Name', 'age', 'Experience', 'Salary']

In [17]:
#selecting specific features
df_pyspark.select('Name', 'age').show()

+---------+---+
|     Name|age|
+---------+---+
|    Krish| 31|
|Sudhanshu| 30|
|    Sunny| 29|
|     Paul| 24|
|   Harsha| 21|
|  Shubham| 23|
+---------+---+



In [18]:
#displaying dataset info
df_pyspark.describe().show()

+-------+------+------------------+-----------------+------------------+
|summary|  Name|               age|       Experience|            Salary|
+-------+------+------------------+-----------------+------------------+
|  count|     6|                 6|                6|                 6|
|   mean|  null|26.333333333333332|4.666666666666667|21333.333333333332|
| stddev|  null| 4.179314138308661|3.559026084010437| 5354.126134736337|
|    min|Harsha|                21|                1|             15000|
|    max| Sunny|                31|               10|             30000|
+-------+------+------------------+-----------------+------------------+



In [22]:
#add cols
df_pyspark=df_pyspark.withColumn('Age after 2 years',df_pyspark['Age']+2)

In [23]:
df_pyspark.show()

+---------+---+----------+------+-----------------+
|     Name|age|Experience|Salary|Age after 2 years|
+---------+---+----------+------+-----------------+
|    Krish| 31|        10| 30000|               33|
|Sudhanshu| 30|         8| 25000|               32|
|    Sunny| 29|         4| 20000|               31|
|     Paul| 24|         3| 20000|               26|
|   Harsha| 21|         1| 15000|               23|
|  Shubham| 23|         2| 18000|               25|
+---------+---+----------+------+-----------------+



In [25]:
#drop cols
df_pyspark=df_pyspark.drop('Age after 2 years')

In [26]:
df_pyspark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [29]:
#rename cols
df_pyspark=df_pyspark.withColumnRenamed('Name','New name')

In [30]:
df_pyspark.show()

+---------+---+----------+------+
| New name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



# Handling Missing Values